In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os

In [2]:
# Конфигурация подключения к PostgreSQL (база в Docker)
DB_CONFIG = {
    'host': 'localhost',  # или IP-адрес Docker контейнера
    'port': '5432',
    'database': 'server_metrics',  # имя вашей базы данных
    'user': 'postgres',
    'password': 'postgres'  # замените на ваш пароль
}

# Путь к Excel файлу
EXCEL_FILE_PATH = '../data/metrics.xlsx'

In [3]:
def create_connection():
    """Создание подключения к базе данных"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("Подключение к базе данных установлено")
        return conn
    except Exception as e:
        print(f"Ошибка подключения к базе данных: {e}")
        return None

In [4]:
def create_table_if_not_exists(conn):
    """Создание таблицы, если она не существует"""
    create_table_query = """
    CREATE TABLE IF NOT EXISTS vm_metrics (
        id SERIAL PRIMARY KEY,
        vm_name VARCHAR(255),
        date DATE,
        metric_name VARCHAR(100),
        max_value DECIMAL(15, 6),
        min_value DECIMAL(15, 6),
        avg_value DECIMAL(15, 6),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    CREATE INDEX IF NOT EXISTS idx_vm_name ON vm_metrics(vm_name);
    CREATE INDEX IF NOT EXISTS idx_date ON vm_metrics(date);
    CREATE INDEX IF NOT EXISTS idx_metric ON vm_metrics(metric_name);
    """

    try:
        cursor = conn.cursor()
        cursor.execute(create_table_query)
        conn.commit()
        print("Таблица создана или уже существует")
        cursor.close()
    except Exception as e:
        print(f"Ошибка при создании таблицы: {e}")

In [ ]:
def read_excel_data(file_path):
    """Чтение данных из Excel файла"""
    try:
        # Читаем Excel файл
        df = pd.read_excel(file_path)
        print(f"Прочитано {len(df)} строк из Excel файла")
        print(f"Колонки: {df.columns.tolist()}")

        # Переименовываем колонки для удобства
        df.columns = ['vm', 'date', 'metric', 'max_value', 'min_value', 'avg_value']

        # Преобразуем дату
        df['date'] = pd.to_datetime(df['date'])

        # Выводим пример данных
        print("\nПример данных:")
        print(df.head())

        return df
    except Exception as e:
        print(f"Ошибка при чтении Excel файла: {e}")
        return None

def insert_data_to_db(conn, df):
    """Вставка данных в базу данных"""
    try:
        # Используем SQLAlchemy для более удобной вставки
        connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(connection_string)

        # Вставляем данные в таблицу
        df.to_sql('vm_metrics', engine, if_exists='append', index=False, method='multi')

        print(f"Успешно вставлено {len(df)} записей в базу данных")

        # Получаем общее количество записей в таблице
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM vm_metrics")
        total_count = cursor.fetchone()[0]
        cursor.close()

        print(f"Всего записей в таблице: {total_count}")

        return True
    except Exception as e:
        print(f"Ошибка при вставке данных в базу: {e}")
        return False

In [ ]:
def insert_data_to_db(conn, df):
    """Вставка данных в базу данных"""
    try:
        # Используем SQLAlchemy для более удобной вставки
        connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(connection_string)

        # Вставляем данные в таблицу
        df.to_sql('vm_metrics', engine, if_exists='append', index=False, method='multi')

        print(f"Успешно вставлено {len(df)} записей в базу данных")

        # Получаем общее количество записей в таблице
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM vm_metrics")
        total_count = cursor.fetchone()[0]
        cursor.close()

        print(f"Всего записей в таблице: {total_count}")

        return True
    except Exception as e:
        print(f"Ошибка при вставке данных в базу: {e}")
        return False

In [ ]:
def clear_table(conn):
    """Очистка таблицы (опционально)"""
    try:
        cursor = conn.cursor()
        cursor.execute("TRUNCATE TABLE vm_metrics RESTART IDENTITY")
        conn.commit()
        cursor.close()
        print("Таблица очищена")
    except Exception as e:
        print(f"Ошибка при очистке таблицы: {e}")

In [ ]:
def main():
    # Проверяем существование файла
    if not os.path.exists(EXCEL_FILE_PATH):
        print(f"Файл {EXCEL_FILE_PATH} не найден!")
        return

    # Создаем подключение к базе
    conn = create_connection()
    if not conn:
        return

    try:
        # Создаем таблицу
        create_table_if_not_exists(conn)

        # Читаем данные из Excel
        df = read_excel_data(EXCEL_FILE_PATH)
        if df is None or df.empty:
            print("Нет данных для загрузки")
            return

        # Опционально: очищаем таблицу перед загрузкой новых данных
        # clear_table(conn)

        # Вставляем данные в базу
        success = insert_data_to_db(conn, df)

        if success:
            print("\nЗагрузка данных успешно завершена!")
        else:
            print("\nЗагрузка данных завершилась с ошибками")

    except Exception as e:
        print(f"Ошибка в основном процессе: {e}")
    finally:
        # Закрываем подключение
        if conn:
            conn.close()
            print("Подключение к базе данных закрыто")